In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [6]:
# Hospital_280_all.csv columns
columns = [
    "id", "record", "hospital_code", "pickup_date",
    "arrival_time", "apot", "impression", "postal_code",
    "agencynumber", "agency_unit", "latitude", "longitude", "status",
    
]

target = ["status"]

In [8]:
# Load the data
file_path = Path('Hospital_280_all.csv')
df = pd.read_csv(file_path, encoding='utf-8')

df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

df = df.drop(["apot"], axis=1)

df.head()



,id,record,hospital_code,pickup_date,arrival_time,impression,postal_code,agencynumber,agency_unit,latitude,longitude,status
0,12,1181,280,2/15/2018,12:50:00,J80,95819,92905,92905-M102,38.57026,-121.452408,0 (< 20 min)
1,28,1391,280,8/12/2018,12:35:00,M79.60,95820,92905,92905-M17,38.57026,-121.452408,0 (< 20 min)
2,53,1899,280,2/26/2019,17:46:00,G89.1,95843,92905,92905-M23,38.57026,-121.452408,0 (< 20 min)
3,92,62394,280,1/2/2018,12:16:00,A41.9,95816,605,605-S-751,38.57026,-121.452408,0 (< 20 min)
4,96,62769,280,1/4/2018,13:50:00,G89.1,95829,605,605-S-751,38.57026,-121.452408,1 (21 - 60 min)


In [9]:
# get data types
df.dtypes

id                 int64
record            object
hospital_code      int64
pickup_date       object
arrival_time      object
impression        object
postal_code        int64
agencynumber       int64
agency_unit       object
latitude         float64
longitude        float64
status            object
dtype: object

In [10]:
# Convert the target column values to low_risk and high_risk based on their values
x = {'0 (< 20 min)': 'benchmark'}   
df = df.replace(x)

x = dict.fromkeys(['1 (21 - 60 min)', '2 (61 - 120 min)', '3 (121 - 180 min)', '4 (181 + min)'], 'extreme')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,id,record,hospital_code,pickup_date,arrival_time,impression,postal_code,agencynumber,agency_unit,latitude,longitude,status
0,12,1181,280,2/15/2018,12:50:00,J80,95819,92905,92905-M102,38.57026,-121.452408,benchmark
1,28,1391,280,8/12/2018,12:35:00,M79.60,95820,92905,92905-M17,38.57026,-121.452408,benchmark
2,53,1899,280,2/26/2019,17:46:00,G89.1,95843,92905,92905-M23,38.57026,-121.452408,benchmark
3,92,62394,280,1/2/2018,12:16:00,A41.9,95816,605,605-S-751,38.57026,-121.452408,benchmark
4,96,62769,280,1/4/2018,13:50:00,G89.1,95829,605,605-S-751,38.57026,-121.452408,extreme


# Split the Data into Training and Testing

In [11]:
# Create our features
X = df.drop(columns='status')
X = pd.get_dummies(X)

# Create our target
y = df[target]

In [12]:
X.describe()

,id,hospital_code,postal_code,agencynumber,latitude,longitude,record_000225abe0e440bb9856c10e993171a0,record_00085af2c55d481898fdbd89b85a4a3f,record_00098461e70d419095ef9430809055d3,record_0014ab2ab6224f2ba4f653d301d3789a,...,agency_unit_90305-R74,agency_unit_90305-TR74,agency_unit_92905-M102,agency_unit_92905-M17,agency_unit_92905-M23,agency_unit_99305-203-05,agency_unit_99305-204-09,agency_unit_99305-215-06,agency_unit_99305-217-07,agency_unit_99305-218-10
count,38514.000000,38514.0,38514.000000,38514.000000,3.851400e+04,3.851400e+04,38514.000000,38514.000000,38514.000000,38514.000000,...,38514.000000,38514.000000,38514.000000,38514.000000,38514.000000,38514.000000,38514.000000,38514.000000,38514.000000,38514.000000
mean,200779.122735,280.0,95808.359947,15013.536636,3.857026e+01,-1.214524e+02,0.000026,0.000026,0.000026,0.000026,...,0.000026,0.000052,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026
std,149390.300229,0.0,361.616661,25490.697625,3.093033e-11,7.875758e-11,0.005096,0.005096,0.005096,0.005096,...,0.005096,0.007206,0.005096,0.005096,0.005096,0.005096,0.005096,0.005096,0.005096,0.005096
min,12.000000,280.0,25818.000000,605.000000,3.857026e+01,-1.214524e+02,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,85231.750000,280.0,95815.000000,1805.000000,3.857026e+01,-1.214524e+02,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,157061.000000,280.0,95821.000000,1805.000000,3.857026e+01,-1.214524e+02,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,309013.250000,280.0,95827.000000,11805.000000,3.857026e+01,-1.214524e+02,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,543811.000000,280.0,95866.000000,99305.000000,3.857026e+01,-1.214524e+02,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
# Check the balance of our target values
y['status'].value_counts()

benchmark    19396
extreme      19118
Name: status, dtype: int64

In [14]:
from sklearn.model_selection import train_test_split

# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

X_train.shape

(28885, 42344)

### Easy Ensemble AdaBoost Classifier

In [20]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier

model = EasyEnsembleClassifier(n_estimators=100, random_state=1)
model.fit(X_train, y_train)

In [39]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5862644313141503

In [40]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[833, 452],
       [578, 637]], dtype=int64)

In [41]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  benchmark       0.59      0.65      0.52      0.62      0.58      0.34      1285
    extreme       0.58      0.52      0.65      0.55      0.58      0.34      1215

avg / total       0.59      0.59      0.58      0.59      0.58      0.34      2500

